In [459]:
import geopandas as gp 
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
from zipfile import ZipFile

## Hard to count 

In [2]:
htc_data = pd.read_excel('data/hard_to_count/pdb2015tract_2010MRR_2017ACS_NY.xlsx', skiprows=5, dtype={'GEOIDtxt' : 'str'})

In [3]:
tracts = gp.read_file('data/hard_to_count/tl_2018_36_tract.shp')

In [4]:
htc_data[htc_data.Stusps10=='NY'].to_csv('data/hard_to_count_NY.csv', index=False)

In [6]:
pd.merge(tracts,htc_data,left_on='GEOID',right_on='GEOIDtxt')

,STATEFP,COUNTYFP,TRACTCE,GEOID_x,NAME,NAMELSAD,MTFCC,FUNCSTAT,ALAND,AWATER,...,Internet,DialUpOnly,Broadband_Any,Cellular,CellularOnly,Broadband_CableFiberOpticDSL,Broadband_CableFiberOpticDSLOnly,Satellite,SatelliteOnly,OtherOnly
0,36,007,012702,36007012702,127.02,Census Tract 127.02,G5020,S,65411230,228281,...,1682,11,1671,964,107,1528,647,85,26,10
1,36,007,012800,36007012800,128,Census Tract 128,G5020,S,12342855,259439,...,1718,3,1715,832,62,1635,877,31,0,0
2,36,007,012900,36007012900,129,Census Tract 129,G5020,S,14480253,63649,...,473,5,468,280,20,438,183,18,5,0
3,36,007,013000,36007013000,130,Census Tract 130,G5020,S,9934434,381729,...,1425,15,1410,739,65,1338,671,16,0,0
4,36,007,013202,36007013202,132.02,Census Tract 132.02,G5020,S,2451349,3681,...,1081,0,1081,578,57,990,449,48,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4907,36,027,640002,36027640002,6400.02,Census Tract 6400.02,G5020,S,2304853,13876,...,24,0,24,12,0,24,8,4,0,0
4908,36,027,130004,36027130004,1300.04,Census Tract 1300.04,G5020,S,24394743,363637,...,529,11,518,348,3,507,165,17,0,0
4909,36,027,050104,36027050104,501.04,Census Tract 501.04,G5020,S,25446931,234551,...,1972,0,1972,1138,38,1934,684,42,0,0
4910,36,119,014500,36119014500,145,Census Tract 145,G5020,S,2778952,14648,...,1767,0,1767,1044,66,1701,723,15,0,0


## Facilities

In [220]:
with ZipFile('data/facilities_201912.csv.zip') as zf:
    facilities  = pd.read_csv(zf.open('facilities_201912.csv'), dtype={'censtract' :'str'})

In [221]:
types=['DAY CARE','EARLY EDUCATION PROGRAM','AFTERSCHOOL PROGRAMS', 'ELEMENTARY SCHOOL - NON-PUBLIC', 'RESIDENTIAL MENTAL HEALTH',
       'ELEMENTARY SCHOOL - PUBLIC','FOOD PANTRY']

facilities =facilities[facilities.factype.isin(types)]
facilities.shape

(6245, 37)

In [222]:
facility_cols = ['facname', 'addressnum', 'streetname', 'address', 'city', 'zipcode',
       'boro', 'borocode', 'bin', 'bbl', 'commboard', 'nta', 'council',
       'schooldist', 'censtract', 'factype','capacity', 'geometry',
       'uid']

In [223]:
facilities_geo = gp.GeoDataFrame(facilities,geometry = facilities.apply(lambda x: Point(x.longitude,x.latitude),axis=1), crs={'init':'EPSG:4326'})
facilities_geo = gp.GeoDataFrame(facilities_geo[facility_cols])


JOIN THESE WITH CENSUS TRACTS AT BOTTOM TO GET PROPER GEOID

In [224]:
facilities[facility_cols + ["latitude","longitude"]].to_csv('../public/facilities.csv', index=False)

In [225]:
facilities.censtract

1           NaN
2        018100
3           NaN
4           NaN
5           NaN
          ...  
34247    023600
34249    025100
34251    009400
34258    039800
34270       NaN
Name: censtract, Length: 6245, dtype: object

## Mail Strategies

In [10]:
mail_strategy_data = pd.read_excel('data/2020-mail-strategies-by-tract.xlsx')

In [11]:
mail_strategy_data.BCUSTATEFP.value_counts().sort_index()


1     1173
2      119
4     1463
5      686
6     7987
8     1233
9      828
10     214
11     179
12    4174
13    1949
15     295
16     295
17    3116
18    1507
19     823
20     765
21    1095
22    1124
23     351
24    1387
25    1468
26    2759
27    1330
28     659
29    1390
30     260
31     528
32     675
33     294
34    1998
35     467
36    4874
37    2160
38     197
39    2943
40    1029
41     825
42    3209
44     240
45    1087
46     203
47    1488
48    5213
49     578
50     183
51    1879
53    1436
54     463
55    1393
56     124
Name: BCUSTATEFP, dtype: int64

In [12]:
mail_strategy_data.LANGUAGE.value_counts()

0    64378
1     7737
Name: LANGUAGE, dtype: int64

In [13]:
new_york_mail_strategy = mail_strategy_data[mail_strategy_data.BCUSTATEFP==36]

In [14]:
new_york_mail_strategy

,CONTACT_STRATEGY,LANGUAGE,BCUSTATEFP,BCUCOUNTYFP,BCUTRACTCE
42591,1,0,36,1,100
42592,2,0,36,1,200
42593,1,0,36,1,300
42594,1,0,36,1,401
42595,1,0,36,1,403
...,...,...,...,...,...
47460,1,0,36,123,150100
47461,1,0,36,123,150200
47462,1,0,36,123,150300
47463,1,0,36,123,150400


In [15]:
new_york_mail_strategy= (new_york_mail_strategy.assign(state  =new_york_mail_strategy.BCUSTATEFP.apply(lambda x :'{0:0>2}'.format(x)),
                              county =  new_york_mail_strategy.BCUCOUNTYFP.apply(lambda x: '{0:0>3}'.format(x)),
                              tract  = new_york_mail_strategy.BCUTRACTCE.apply(lambda x: '{0:0>6}'.format(x))
                             )
                      .assign(geoid = lambda x: x.state + x.county + x.tract))

new_york_mail_strategy = new_york_mail_strategy.set_index('geoid')


In [16]:
new_york_mail_strategy

,CONTACT_STRATEGY,LANGUAGE,BCUSTATEFP,BCUCOUNTYFP,BCUTRACTCE,state,county,tract
geoid,,,,,,,,
36001000100,1,0,36,1,100,36,001,000100
36001000200,2,0,36,1,200,36,001,000200
36001000300,1,0,36,1,300,36,001,000300
36001000401,1,0,36,1,401,36,001,000401
36001000403,1,0,36,1,403,36,001,000403
...,...,...,...,...,...,...,...,...
36123150100,1,0,36,123,150100,36,123,150100
36123150200,1,0,36,123,150200,36,123,150200
36123150300,1,0,36,123,150300,36,123,150300


In [50]:
new_york_mail_strategy.LANGUAGE.value_counts()

0    4202
1     672
Name: LANGUAGE, dtype: int64

## Geojson

In [79]:
htc_data.

11

In [165]:
htc_data[htc_data.GEOIDtxt==36047000501]

,GEOID,GEOIDtxt,State,Stusps10,State_name,County10,County_name10,Tract10,MRR2010,MRR20pctthreshold,...,Internet,DialUpOnly,Broadband_Any,Cellular,CellularOnly,Broadband_CableFiberOpticDSL,Broadband_CableFiberOpticDSLOnly,Satellite,SatelliteOnly,OtherOnly
1097,36047000501,36047000501,36,NY,New York,47,Kings County,5.01,85.6,0,...,1440,38,1402,901,36,1366,501,0,0,0


In [159]:
htc_data[htc_data.County_name10=='Kings County']

,GEOID,GEOIDtxt,State,Stusps10,State_name,County10,County_name10,Tract10,MRR2010,MRR20pctthreshold,...,Internet,DialUpOnly,Broadband_Any,Cellular,CellularOnly,Broadband_CableFiberOpticDSL,Broadband_CableFiberOpticDSLOnly,Satellite,SatelliteOnly,OtherOnly
1094,36047000100,36047000100,36,NY,New York,47,Kings County,1.00,84.6,0,...,1843,20,1823,1110,79,1715,690,83,0,0
1095,36047000200,36047000200,36,NY,New York,47,Kings County,2.00,62.7,1,...,231,0,231,118,38,193,89,31,0,0
1096,36047000301,36047000301,36,NY,New York,47,Kings County,3.01,84.3,0,...,1739,10,1729,1231,119,1610,457,44,0,0
1097,36047000501,36047000501,36,NY,New York,47,Kings County,5.01,85.6,0,...,1440,38,1402,901,36,1366,501,0,0,0
1098,36047000502,36047000502,36,NY,New York,47,Kings County,5.02,85.6,0,...,1453,19,1434,960,35,1399,464,17,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,36047122000,36047122000,36,NY,New York,47,Kings County,1220.00,59.8,1,...,1868,0,1868,635,164,1675,1137,119,29,0
1851,36047123700,36047123700,36,NY,New York,47,Kings County,1237.00,59.4,1,...,612,0,612,312,84,528,271,29,0,0
1852,36047150200,36047150200,36,NY,New York,47,Kings County,1502.00,73.0,1,...,917,5,912,423,31,881,471,27,0,0
1853,36047152200,36047152200,36,NY,New York,47,Kings County,1522.00,73.0,1,...,1003,0,1003,596,62,900,348,73,22,0


In [163]:
census_tracts[census_tracts.boro_name=='Brooklyn']

,ntacode,shape_area,ntaname,shape_leng,boro_name,puma,ct2010,boro_code,cdeligibil,boro_ct2010,ctlabel,geometry,ct_county,CONTACT_STRATEGY,LANGUAGE
geoid,,,,,,,,,,,,,,,
36047000501,BK09,901506.992481,Brooklyn Heights-Cobble Hill,4828.17004215,Brooklyn,4004,000501,3,I,3000501,5.01,"MULTIPOLYGON (((-73.99302 40.69746, -73.99379 ...",047,1,0
36047004100,BK38,1744094.3952,DUMBO-Vinegar Hill-Downtown Brooklyn-Boerum Hill,5283.85665969,Brooklyn,4004,004100,3,I,3004100,41,"MULTIPOLYGON (((-73.98182 40.68733, -73.98223 ...",047,1,0
36047005800,BK31,2080303.31132,Bay Ridge,6098.16780024,Brooklyn,4013,005800,3,E,3005800,58,"MULTIPOLYGON (((-74.02956 40.61983, -74.02821 ...",047,1,0
36047006000,BK31,1639074.04175,Bay Ridge,5498.33438728,Brooklyn,4013,006000,3,I,3006000,60,"MULTIPOLYGON (((-74.02835 40.62280, -74.02864 ...",047,1,0
36047006700,BK33,1680804.38927,Carroll Gardens-Columbia Street-Red Hook,5236.04791182,Brooklyn,4005,006700,3,I,3006700,67,"MULTIPOLYGON (((-73.99441 40.68545, -73.99458 ...",047,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36047016000,BK31,2785264.01265,Bay Ridge,7935.13018528,Brooklyn,4013,016000,3,E,3016000,160,"MULTIPOLYGON (((-74.02621 40.62149, -74.02536 ...",047,1,0
36047016200,BK31,2621970.92342,Bay Ridge,8493.77995794,Brooklyn,4013,016200,3,I,3016200,162,"MULTIPOLYGON (((-74.02615 40.61626, -74.02634 ...",047,1,0
36047016400,BK31,8407411.10496,Bay Ridge,16774.4313677,Brooklyn,4013,016400,3,I,3016400,164,"MULTIPOLYGON (((-74.02575 40.61606, -74.02607 ...",047,1,0


In [150]:
htc_data.set_index("GEOIDtxt")

,GEOID,State,Stusps10,State_name,County10,County_name10,Tract10,MRR2010,MRR20pctthreshold,UE_flag,...,Internet,DialUpOnly,Broadband_Any,Cellular,CellularOnly,Broadband_CableFiberOpticDSL,Broadband_CableFiberOpticDSLOnly,Satellite,SatelliteOnly,OtherOnly
GEOIDtxt,,,,,,,,,,,,,,,,,,,,,
36001000100,36001000100,36,NY,New York,1,Albany County,1.00,67.9,1,0,...,683,0,683,389,106,574,286,35,3,0
36001000200,36001000200,36,NY,New York,1,Albany County,2.00,60.4,1,0,...,1410,15,1395,790,185,1182,605,7,0,0
36001000300,36001000300,36,NY,New York,1,Albany County,3.00,73.0,1,0,...,1857,19,1838,791,175,1663,994,9,0,0
36001000401,36001000401,36,NY,New York,1,Albany County,4.01,88.1,0,0,...,808,0,808,344,29,779,433,0,0,0
36001000403,36001000403,36,NY,New York,1,Albany County,4.03,76.9,0,0,...,1978,0,1978,923,122,1824,1039,16,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36123150100,36123150100,36,NY,New York,123,Yates County,1501.00,90.2,0,0,...,1710,6,1704,910,162,1385,685,165,56,8
36123150200,36123150200,36,NY,New York,123,Yates County,1502.00,84.6,0,0,...,1227,22,1205,674,82,998,498,118,19,14
36123150300,36123150300,36,NY,New York,123,Yates County,1503.00,90.7,0,0,...,1534,30,1504,808,182,1145,522,218,132,7


In [17]:
census_tracts = gp.read_file('../public/2010 Census Tracts.geojson')
def strategy_code(x):
    if( x.CONTACT_STRATEGY==1 and x.LANGUAGE == 0):
        return 0
    if(x.CONTACT_STRATEGY==1 and x.LANGUAGE == 1):
        return 1
    if(x.CONTACT_STRATEGY==2 and x.LANGUAGE == 0):
        return 2
    if(x.CONTACT_STRATEGY==2 and x.LANGUAGE == 1):
        return 3
    
    
def boro_to_county(boro_code):
    if boro_code =='1':
        return '061'
    if boro_code =='2':
        return '005'
    if boro_code =='3':
        return '047'
    if boro_code =='4':
        return '081'
    if boro_code =='5':
        return '085'
    
census_tracts= census_tracts.assign(ct_county = census_tracts.boro_code.apply(lambda x : boro_to_county(x)))
census_tracts = census_tracts.assign(geoid = '36' + census_tracts.ct_county + census_tracts.ct2010 ).set_index('geoid')

census_tracts= census_tracts.join(new_york_mail_strategy[ ['CONTACT_STRATEGY',"LANGUAGE"]],how='inner')
census_tracts = census_tracts.join(htc_data.set_index('GEOIDtxt'), how='inner')
census_tracts= census_tracts.assign(strategy_code = census_tracts.apply(lambda x: strategy_code(x),axis=1))
census_tracts.plot(column="CONTACT_STRATEGY")

In [18]:
census_tracts.to_file('../public/CensusTractsAllVariables.geojson', driver ='GeoJSON', `)

In [526]:
import json
import geopandas as gp

In [528]:
def addID(feature):
    feature['id'] = feature['properties']['GEOID']
    return feature
with open("../public/census_tracts.geojson",'r') as f:
    tracts = json.load(f)
    tracts['features'] =[ addID(feature) for feature in tracts['features']]

In [529]:
with open('../public/census_tracts.geojson','w') as f:
    json.dump(tracts,f)

In [171]:
census_tracts.CONTACT_STRATEGY.value_counts()

1    1446
2     698
Name: CONTACT_STRATEGY, dtype: int64

facilities

In [235]:
gp.sjoin(facilities_geo,census_tracts)[facility_cols+['GEOID']].to_file('../public/facilities.geojson', driver='GeoJSON')

/home/jovyan/.local/lib/python3.7/site-packages/geopandas/tools/sjoin.py:61: UserWarning: CRS of frames being joined does not match!(None != {'init': 'epsg:4326'})
  "(%s != %s)" % (left_df.crs, right_df.crs)


## New Data

In [417]:
tracts = gp.read_file('../public/CensusTractsAllVariables.geojson')

In [418]:
import pandas as pd 
import numpy as np

In [419]:
total_pop = pd.read_csv('NewData/ACSDT5Y2018.B01003_TotalPopulation.csv' ,skiprows=1, dtype={'id':str})
total_pop = total_pop.rename(columns={'Estimate!!Total':'total_population'})[['id','total_population']]
total_pop['GEOID'] = total_pop['id'].str.split("US").str[-1].astype('int64')
total_pop = total_pop.drop('id',axis=1)

In [420]:
new_tracts = total_pop.set_index("GEOID").join(tracts[['GEOID','MRR2010','MRR20pctthreshold','strategy_code']].set_index('GEOID'),how='inner')

In [422]:
race = pd.read_csv('NewData/ACSDT5Y2018.B02001_Race_NYC_Updated.csv' ,skiprows=1, dtype={'id':str})
race = race.rename(columns={
  'Estimate!!Total' : 'race_total',
  'Estimate!!Total!!White alone': 'race_white',
  'Estimate!!Total!!Black or African American alone' : 'race_black',
  'Estimate!!Total!!American Indian and Alaska Native alone': 'race_native_american',
  'Estimate!!Total!!Asian alone': 'race_asian',
  'Estimate!!Total!!Native Hawaiian and Other Pacific Islander alone' : 'race_hawaiian',
 'Estimate!!Total!!Some other race alone' : 'race_other'
}).drop(['Geographic Area Name','TOTAL: Other Populations'],axis=1)

race = race.assign(race_hispanic = race.race_total  - race[['race_black','race_native_american','race_asian','race_hawaiian','race_other']].sum(axis=1))
race = race.assign(GEOID=race.id.astype('int64')).drop('id',axis=1).set_index('GEOID')
new_tracts = new_tracts.join(race,how='inner')


In [424]:
foreign_born =  pd.read_csv('NewData/ACSDT5Y2018.B05002_ForeignBorn_NYC_Updated.csv' ,skiprows=1, dtype={'id':str})
foreign_born = foreign_born.rename(columns={ 'Estimate!!Total': 'foreign_born_total',
                                            'Estimate!!Total!!Native' : 'foreign_born_native',
                                            'Estimate!!Total!!Foreign born': 'foreign_born_foreign'
                                           }).drop(['Geographic Area Name'],axis=1)
foreign_born=foreign_born.assign(GEOID=foreign_born.id.astype('int64')).set_index('GEOID').drop("id",axis=1)
new_tracts = new_tracts.join(foreign_born,how='inner')


In [426]:
own_vs_rent = pd.read_csv("NewData/ACSDT5Y2018.B25003_Tenure_OwnVSRent.csv",skiprows=1,dtype={'id':str})
own_vs_rent = own_vs_rent.rename(columns={'Estimate!!Total':'own_vs_rent_total', 
                            'Estimate!!Total!!Owner occupied':'own_vs_rent_owner',
                            'Estimate!!Total!!Renter occupied':'own_vs_rent_rent'}).drop(['Geographic Area Name'],axis=1)
own_vs_rent = own_vs_rent.assign(GEOID=own_vs_rent.id.str.split('US').str[-1].astype('int64')).drop('id',axis=1).set_index("GEOID")
own_vs_rent = own_vs_rent[['own_vs_rent_total','own_vs_rent_owner','own_vs_rent_rent']]
new_tracts = new_tracts.join(own_vs_rent,how='inner')


In [428]:
avg_household_size = pd.read_csv('NewData/ACSDT5Y2018.B25010_AvgHouseholdSize.csv',skiprows=1,dtype={'id':str})
avg_household_size = (avg_household_size.rename(columns= { 'Estimate!!Total' : 'hosehold_size_total',
                                                        'Estimate!!Total!!Owner occupied' : 'household_size_owner_occupied',
                                                        'Estimate!!Total!!Renter occupied' : 'household_size_renter_occupied'})
                                                .drop(['Geographic Area Name', 'Margin of Error!!Total', 
                                                       'Margin of Error!!Total!!Owner occupied',
                                                       'Margin of Error!!Total!!Renter occupied'],axis=1))
avg_household_size = avg_household_size.assign(GEOID=avg_household_size.id.str.split("US").str[-1].astype('int64')).drop('id',axis=1).set_index("GEOID")
new_tracts = new_tracts.join(avg_household_size,how='inner')


In [410]:
# rent_burden = pd.read_csv('NewData/ACSDT5Y2018.B25074_RentBurden.csv', skiprows=1, dtype={'id':str})
# rent_burden = rent_burden.rename(columns={'Estimate!!Total' : 'rent_burden_total',
#                                          'Estimate!!Total!!Less than $10,000' : 'rent_burden_less_10_000_total',
#                                          'Estimate!!Total!!Less than $10,000!!Less than 20.0 percent',: 'rent_burden_less_10_000_less_20pc',
#                                          })


In [430]:
internet = pd.read_csv('NewData/ACSDT5Y2018.B28002_InternetSubscriptions_Updated.csv', skiprows=1, dtype={'id':str})
internet = internet.rename(columns={'Estimate!!Total' : 'internet_total', 'Estimate!!Total!!With an Internet subscription': 'internet_subscription',
                        'Estimate!!Total!!Internet access without a subscription': 'internet_no_subscription',
                       'Estimate!!Total!!No Internet access' : 'internet_no_access' }).drop(['Geographic Area Name'],axis=1)
internet = internet.assign(GEOID=internet.id.str.split("US").str[-1].astype('int64')).drop('id',axis=1).set_index("GEOID")
new_tracts = new_tracts.join(internet,how='inner')


In [432]:
computer = pd.read_csv("NewData/ACSDT5Y2018.B28003_HasAComputer.csv",skiprows=1,dtype={"id":str})
cols = [ col for col in computer if '!'in col and not 'Margin' in col]
new_cols = ["computer_{}".format(col.split("!!")[-1].lower().replace(" ","_")  ) for col in cols]
computer = computer.rename(columns= dict(zip(cols,new_cols)))[['id']+new_cols]

In [434]:
age = pd.read_csv("NewData/ACSST5Y2018.S0101_Age_NYC.csv", skiprows=1,dtype={"id":str})
age = age.assign(age_total = age['Estimate!!Total!!Total population'],
          age_less_5 = age['Estimate!!Total!!Total population!!AGE!!Under 5 years'],
          age_6_15 = age['Estimate!!Total!!Total population!!AGE!!5 to 9 years']+ age['Estimate!!Total!!Total population!!AGE!!10 to 14 years'] ,
          age_16_64 = age[['Estimate!!Total!!Total population!!AGE!!10 to 14 years',
                           'Estimate!!Total!!Total population!!AGE!!15 to 19 years',
                          'Estimate!!Total!!Total population!!AGE!!20 to 24 years',
                            'Estimate!!Total!!Total population!!AGE!!25 to 29 years',
                          'Estimate!!Total!!Total population!!AGE!!30 to 34 years',
                            'Estimate!!Total!!Total population!!AGE!!35 to 39 years',
                            'Estimate!!Total!!Total population!!AGE!!40 to 44 years',
                            'Estimate!!Total!!Total population!!AGE!!45 to 49 years',
                            'Estimate!!Total!!Total population!!AGE!!50 to 54 years',
                            'Estimate!!Total!!Total population!!AGE!!55 to 59 years',
                            'Estimate!!Total!!Total population!!AGE!!60 to 64 years']].sum(axis=1),
         age_64_over = age[['Estimate!!Total!!Total population!!AGE!!65 to 69 years',
       'Estimate!!Total!!Total population!!AGE!!70 to 74 years',
       'Estimate!!Total!!Total population!!AGE!!75 to 79 years',
       'Estimate!!Total!!Total population!!AGE!!80 to 84 years',
       'Estimate!!Total!!Total population!!AGE!!85 years and over']].sum(axis=1),
           
        GEOID = age.id.astype('int64')
         ).set_index('GEOID')[['age_total','age_less_5','age_6_15','age_16_64','age_64_over']]
new_tracts=new_tracts.join(age,how='inner')

In [436]:
income = pd.read_csv('NewData/ACSST5Y2018.S1901_IncomeHouseholds_NYC.csv',skiprows=1,dtype={
    "id":'int64'})
income = income.assign(median_income = income['Estimate!!Households!!Median income (dollars)'],
              pc_below_poverty = income[['Estimate!!Households!!Total!!Less than $10,000',
       'Estimate!!Households!!Total!!$10,000 to $14,999',
       'Estimate!!Households!!Total!!$15,000 to $24,999']].apply(pd.to_numeric,  errors='coerce').sum(axis=1))

income = income.assign(income_houses_total = income['Estimate!!Households!!Total'],
                      homes_below_poverty = income.pc_below_poverty*income['Estimate!!Households!!Total']/100.0)

income = income.assign(GEOID=income['id']).set_index('GEOID')
new_tracts= new_tracts.join(income[['median_income','pc_below_poverty','income_houses_total','homes_below_poverty']],how='inner')

In [438]:
english_proficency = pd.read_csv("NewData/ACSST5Y2018.S1602_LimitedEnglishProficiency.csv", skiprows=1,dtype={"id":str})
english_proficency = english_proficency.assign(GEOID=english_proficency.id.str.split('US').str[-1].astype('int64')).drop('id',axis=1)
english_proficency = english_proficency.assign( english_total_households = english_proficency['Estimate!!Total!!All households'], 
                           english_spanish = english_proficency['Estimate!!Total!!All households!!Households speaking --!!Spanish'],
                           english_european = english_proficency['Estimate!!Total!!All households!!Households speaking --!!Other Indo-European languages'],
                           english_asian  = english_proficency['Estimate!!Limited English-speaking households!!All households!!Households speaking --!!Asian and Pacific Island languages'],
                           english_other  = english_proficency['Estimate!!Total!!All households!!Households speaking --!!Other languages']
                         )
english_proficency = english_proficency.assign(english_english =  
                                               english_proficency.english_total_households - english_proficency[['english_spanish','english_european','english_asian','english_other']].sum(axis=1)
                                              )
english_proficency= english_proficency[[
    'GEOID',
    'english_total_households',
    'english_spanish',
    'english_european',
    'english_asian',
    'english_other',
    'english_english']].set_index('GEOID')

new_tracts= new_tracts.join(english_proficency,how='inner')

In [440]:
all_joined = gp.GeoDataFrame(new_tracts.join(tracts.set_index('GEOID')[['geometry']],how='inner').reset_index().rename(columns={'index':"GEOID"}))

In [524]:
all_joined = all_joined.assign(gid=all_joined.GEOID).set_index('gid')

In [525]:
all_joined.to_file('../public/census_tracts.geojson',driver='GeoJSON')

## Other boundaries

In [362]:
community_districts = gp.read_file('../public/boundaries/cd.geojson')
city_council_districts =gp.read_file('../public/boundaries/cc.geojson')
school_districts = gp.read_file('../public/boundaries/sd.geojson')
ntas = gp.read_file('../public/boundaries/nta.geojson')

DriverError: '../public/boundaries/nta.geojson' not recognized as a supported file format.

In [451]:
all_joined.columns

Index(['GEOID', 'total_population', 'MRR2010', 'MRR20pctthreshold',
       'strategy_code', 'race_total', 'race_white', 'race_black',
       'race_native_american', 'race_asian', 'race_hawaiian', 'race_other',
       'race_hispanic', 'foreign_born_total', 'foreign_born_native',
       'foreign_born_foreign', 'own_vs_rent_total', 'own_vs_rent_owner',
       'own_vs_rent_rent', 'hosehold_size_total',
       'household_size_owner_occupied', 'household_size_renter_occupied',
       'internet_total', 'internet_subscription', 'internet_no_subscription',
       'internet_no_access', 'age_total', 'age_less_5', 'age_6_15',
       'age_16_64', 'age_64_over', 'median_income', 'pc_below_poverty',
       'income_houses_total', 'homes_below_poverty',
       'english_total_households', 'english_spanish', 'english_european',
       'english_asian', 'english_other', 'english_english', 'geometry'],
      dtype='object')

In [452]:
def generate_boundary_level(boundary,census_tracts,boundary_id):
    intersection = gp.overlay(census_tracts,boundary).assign(area=lambda x:x.area)
    intersection = (intersection.set_index('GEOID').assign(tract_area = census_tracts.set_index("GEOID").area)
                                   .assign(tract_fraction = lambda x: x.area/x.tract_area))
    
    intersection = intersection.rename(columns={boundary_id:'geoid'})
    a
    addative_vars = ['total_population', 'MRR2010', 'MRR20pctthreshold',
           'race_total', 'race_white', 'race_black',
       'race_native_american', 'race_asian', 'race_hawaiian', 'race_other',
       'race_hispanic', 'foreign_born_total', 'foreign_born_native',
       'foreign_born_foreign', 'own_vs_rent_total', 'own_vs_rent_owner',
       'own_vs_rent_rent', 'hosehold_size_total',
       'household_size_owner_occupied', 'household_size_renter_occupied',
       'internet_total', 'internet_subscription', 'internet_no_subscription',
       'internet_no_access', 'age_total', 'age_less_5', 'age_6_15',
       'age_16_64', 'age_64_over', 'median_income', 'pc_below_poverty',
       'income_houses_total', 'homes_below_poverty',
       'english_total_households', 'english_spanish', 'english_european',
       'english_asian', 'english_other', 'english_english' ]

    addative = intersection[addative_vars].apply(pd.to_numeric,errors='coerce').apply(lambda x: x * intersection.tract_fraction)
    addative = addative.assign(geoid = intersection.geoid)
    addative = addative.groupby('geoid').sum()
    addative = gp.GeoDataFrame(addative.join(boundary.rename(columns={boundary_id:'geoid'}).set_index('geoid')[['geometry']]))
    addative = addative.reset_index()
    return addative

In [453]:
community_districts_with_vars = generate_boundary_level(community_districts,all_joined,'boro_cd')
city_council_district_with_vars = generate_boundary_level(city_council_districts,all_joined,'coun_dist')
school_districts_with_vars = generate_boundary_level(school_districts,all_joined,'school_dist')
ntas_with_vars = generate_boundary_level(ntas,all_joined,'ntacode')

community_districts_with_vars.to_file('../public/boundaries/community_districts_vars.geojson',driver='GeoJSON')
city_council_district_with_vars.to_file('../public/boundaries/city_council_district_with_vars.geojson',driver='GeoJSON')
school_districts_with_vars.to_file('../public/boundaries/school_districts_with_vars.geojson',driver='GeoJSON')
ntas_with_vars.to_file('../public/boundaries/ntas_with_vars.geojson',driver='GeoJSON')

In [ ]:
community_districts_with_vars.to_file('../public/boundaries/community_districts_vars.geojson',driver='GeoJSON')

In [292]:
def generate_labels(boundary,id_col):
    result = boundary.rename(columns={id_col:'geoid'}) 
    result = result.assign(geometry=result.geometry.centroid)
    return result

In [324]:
community_district_labels = generate_labels(community_districts,'boro_cd')
city_council_district_labels = generate_labels(city_council_districts,'coun_dist')
school_district_labels = generate_labels(school_districts,'school_dist')
nta_labels = generate_labels(ntas,'ntacode')

community_district_labels.to_file('../public/boundaries/communty_district_labels.geojson',driver='GeoJSON')
city_council_district_labels.to_file('../public/boundaries/city_council_district_labels.geojson',driver='GeoJSON')
school_district_labels.to_file('../public/boundaries/school_district_labels.geojson',driver='GeoJSON')
nta_labels.to_file('../public/boundaries/nta_labels.geojson',driver='GeoJSON')

## Dot layers

In [455]:
!ls NewData/

 ACSDT5Y2018.B01003_TotalPopulation.csv
 ACSDT5Y2018.B02001_Race_NYC.csv
 ACSDT5Y2018.B02001_Race_NYC_Updated.csv
 ACSDT5Y2018.B05002_ForeignBorn_NYC.csv
 ACSDT5Y2018.B05002_ForeignBorn_NYC_Updated.csv
 ACSDT5Y2018.B25003_Tenure_OwnVSRent.csv
 ACSDT5Y2018.B25010_AvgHouseholdSize.csv
 ACSDT5Y2018.B25074_RentBurden.csv
 ACSDT5Y2018.B28002_InternetSubscriptions.csv
 ACSDT5Y2018.B28002_InternetSubscriptions_Updated.csv
 ACSDT5Y2018.B28003_HasAComputer.csv
 ACSST5Y2018.S0101_Age_NYC.csv
 ACSST5Y2018.S1601_LanguageSpokenAtHome.csv
 ACSST5Y2018.S1602_LimitedEnglishProficiency.csv
 ACSST5Y2018.S1810_Disability.csv
 ACSST5Y2018.S1810_Disability_Updated.csv
 ACSST5Y2018.S1901_IncomeHouseholds_NYC.csv
'Community Assets (Dot Layers)'
'Community Assets (Dot Layers)-20200210T171153Z-001.zip'
'Interactive Map Demographic Data Dictionary.xlsx'
 reconferencelineforourcallat11.zip


In [504]:
dot_layer_path = Path('NewData/dot_layers/')

In [505]:
dot_layers = {
    "HospitalsandHealthClinics_Layer":"Hospitals and Health Clinics",
    "FaithBasedOrganizations_Layer":"Faith Based Organizations",
    "CityWide Community Schools":"City Wide Community Schools",
    "LGBTQOrganizations_Layer":"LGBTQ Organizations",
    "Food_kitchens_and_food_pantries":"Food kitchens and food pantries",
    "Senior_Centers":"Senior Centers",
    "Universal_PreK":"Universal PreK",
    "PUBLIC_K12": "K12",
    "Community_centers_and_community_schools":"Community centers and community schools",
    "MentalHealthOrganizations_Layer":"Mental Health Organizations Layer",
    "CommunityBasedOrganizations_Layer":"Community Based Organizations",
    "Public_Libraries":"Public Libraries"
}

In [518]:
all_point_data = []
for layer_name in dot_layers.keys():
    point_data = gp.read_file( dot_layer_path / ( layer_name + '.shp') )
    for col in point_data.columns:
        print(col)
    print('\n')
    all_point_data.append(point_data.assign(poi_type = dot_layers[layer_name]))
all_point_data = pd.concat(all_point_data)


Facility_T
Borough
Facility_N
Cross_Stre
Phone
Location_1
Postcode
Latitude
Longitude
Community
Council_Di
Census_Tra
BIN
BBL
NTA
Latitude_1
Longitud_1
Accuracy_S
Accuracy_T
Number
Street
City
State
County
Zip
Country
Source
geometry


Partner
First_Name
Last_Name
Email
Street
City
State
Zip
Latitude
Longitude
Accuracy_S
Accuracy_T
Number
Street_1
City_1
State_1
County
Zip_1
Country
Source
geometry


address
addressnum
agencyjuri
area
areatype
bbl
bin
boro
borocode
capacity
captype
censtract
city
commboard
council
datadate
dataname
datasource
dataurl
facdomain
facgroup
facname
facsubgrp
factype
idagency
idold
latitude
longitude
nta
opabbrev
opname
optype
overabbrev
overagency
overlevel
pgtable
proptype
streetname
uid
uid_merged
util
utilrate
xcoord
ycoord
zipcode
geometry


Organizati
Street
City
State
Zip
Latitude
Longitude
Accuracy_S
Accuracy_T
Number
Street_1
City_1
State_1
County
Zip_1
Country
Source
geometry


address
addressnum
agencyjuri
area
areatype
bbl
bin
boro
borocode
capac

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


0                 Ida G. Israel Community Health Center
1                  South Queens Community Health Center
2                    Melrose Houses Child Health Clinic
3             Daniel Webster Houses Child Health Clinic
4     Segundo Ruiz Belvis Diagnostic & Treatment Center
                            ...                        
73                     Bushwick Community Health Center
74                             Bellevue Hospital Center
75                         Ridgewood Communicare Clinic
76                      Smith Communicare Health Center
77             Morrisania Diagnostic & Treatment Center
Name: Facility_N, Length: 78, dtype: object